Setting up the logistic regression

In [51]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [52]:
googFile = 'C:/Users/pdeov/OneDrive/Documents/GitHub/algorithmic-trading-python/logistic_regression/GOOG.csv'
spFile = 'C:/Users/pdeov/OneDrive/Documents/GitHub/algorithmic-trading-python/logistic_regression/S&P500.csv'

In [58]:
goog = pd.read_csv(googFile, sep=",", usecols=[0,5], names=['Date','Goog'],header=0)
sp = pd.read_csv(spFile, sep=",", usecols=[0,5], names=['Date','SP500'],header=0)

In [59]:
goog['SP500'] = sp['SP500']
goog['Date'] = pd.to_datetime(goog['Date'], format= '%d/%m/%Y')

In [60]:
goog.head()

,Date,Goog,SP500
0,2021-01-04,1728.239990,3700.65
1,2021-01-05,1740.920044,3726.86
2,2021-01-06,1735.290039,3748.14
3,2021-01-07,1787.250000,3803.79
4,2021-01-08,1807.209961,3824.68


Converting prices to Returns

In [61]:
goog = goog.sort_values(['Date'], ascending = [True])

In [57]:
goog.head()

,Date,Goog,SP500
0,2021-01-04,1728.239990,3962.71
1,2021-01-05,1740.920044,3968.94
2,2021-01-06,1735.290039,3943.34
3,2021-01-07,1787.250000,3939.34
4,2021-01-08,1807.209961,3898.81


In [66]:
dict(goog.dtypes)

{'Date': dtype('<M8[ns]'), 'Goog': dtype('float64'), 'SP500': dtype('float64')}

In [68]:
[key for key in dict(goog.dtypes) if dict(goog.dtypes)[key] in ['float64','float64']]

['Goog', 'SP500']

In [70]:
goog[[key for key in dict(goog.dtypes) if dict(goog.dtypes)[key] in ['float64','float64']]].pct_change()

,Goog,SP500
0,NaN,NaN
1,0.007337,0.007083
2,-0.003234,0.005710
3,0.029943,0.014847
4,0.011168,0.005492
5,-0.022405,-0.006555
6,-0.011417,0.000416
7,0.004495,0.002276
8,-0.008105,-0.003753
9,-0.002293,-0.007190


In [80]:
returns = goog[[key for key in dict(goog.dtypes) if dict(goog.dtypes)[key] in ['float64','float64']]].pct_change()

In [81]:
returns['Date'] = goog['Date']

In [82]:
#returns["Ïntercept"] = 1

In [83]:
returns[["Goog","SP500"]]

,Goog,SP500
0,NaN,NaN
1,0.007337,0.007083
2,-0.003234,0.005710
3,0.029943,0.014847
4,0.011168,0.005492
5,-0.022405,-0.006555
6,-0.011417,0.000416
7,0.004495,0.002276
8,-0.008105,-0.003753
9,-0.002293,-0.007190


In [99]:
xData = np.array(returns[["Goog","SP500"]][1:-1])

In [100]:
yData = (returns["Goog"] > 0)[2:] #continuous to categorical

In [101]:
logit = sm.Logit(yData, xData)

In [102]:
result = logit.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.674633
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   Goog   No. Observations:                   48
Model:                          Logit   Df Residuals:                       46
Method:                           MLE   Df Model:                            1
Date:                Wed, 17 Mar 2021   Pseudo R-squ.:               -0.004986
Time:                        12:43:17   Log-Likelihood:                -32.382
converged:                       True   LL-Null:                       -32.222
Covariance Type:            nonrobust   LLR p-value:                     1.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1           -16.3257     19.662     -0.830      0.406     -54.863      22.212
x2            -6.2390     39.442     -0.158      0.874     -83.544      71.066
==============================================================================
"""

In [103]:
result.predict(xData)

array([0.45909919, 0.50429288, 0.35859303, 0.44606334, 0.60028879,
       0.54581855, 0.47812035, 0.53885725, 0.52056152, 0.36242919,
       0.27638617, 0.49009812, 0.48355262, 0.49790273, 0.46404983,
       0.71019795, 0.41359968, 0.5891174 , 0.33545061, 0.42278972,
       0.229037  , 0.49825301, 0.42400281, 0.49836104, 0.52005698,
       0.47730183, 0.49641362, 0.47666134, 0.46643088, 0.48817214,
       0.52816346, 0.53380133, 0.58175537, 0.4862235 , 0.43475638,
       0.6569961 , 0.49635851, 0.37605955, 0.52370398, 0.61488921,
       0.47587676, 0.35542156, 0.66522922, 0.42106173, 0.48596496,
       0.36831582, 0.59908649, 0.48083737])

In [105]:
zip(yData, result.predict(xData) > 0.5)